In [1]:
################################################
#
# This notebook provides functions to calculate the 
# coefficients "fd" used in the (p+q):p mean motion 
# resonance width expressions derived in Chapter 6
# of Murray & Dermott's Solar System Dynamics.
# https://ui.adsabs.harvard.edu/abs/1999ssd..book.....M/abstract
# All expressions below for the expansions fo the
# disturbing functiom are taken from Appendix D
#
################################################

import numpy as np
import scipy.integrate as integrate

In [2]:
#Definitions for the Laplace coefficients and their derivatives

def b_integrand(x,j,s,a):
    return np.cos(j*x)/((1.-2.*a*np.cos(x)+a*a)**s)

def b(j,s,a):
    #j = np.abs(j)
    x = integrate.quad(b_integrand, 0., 2.*np.pi, args=(j,s,a))
    return x[0]/(np.pi)

def Db(j,s,a):
    t1 = b(j-1.,s+1.,a)
    t2 = -2.*a*b(j,s+1.,a)
    t3 = b(j+1.,s+1.,a)
    val = s*(t1 + t2 + t3)
    return val

def Dnb(n,j,s,a):
    if(n==2):
        t1 = Db(j-1.,s+1.,a)
        t2 = -2*a*Db(j,s+1.,a)
        t3 = Db(j+1.,s+1.,a)
        t4 = -2.*b(j,s+1.,a)
    elif(n>2):
        t1 = Dnb(n-1,j-1.,s+1.,a)
        t2 = -2*a*Dnb(n-1,j,s+1.,a)
        t3 = Dnb(n-1,j+1.,s+1.,a)
        if(n==3):
            t4 = -2.*(float(n)-1.)*Db(j,s+1,a)
        else:
            t4 = -2.*(float(n)-1.)*Dnb(n-2,j,s+1,a)
    val = s*(t1+t2+t3+t4)
    return val
    

In [11]:
# Resonances are defined as (p+q):p 
# The variable pq is (p+q)
# To return the coefficient fd for an 
# internal (p+q):p, one calls
# fd = fd_int(pq,p)
# To return the coefficient fd for an
# external (p+q):p, one calls
# fd = fd_ext(pq,p)


def fd_int(pq,p):
    a = np.power((float(p)/float(pq)),(2./3))
    q = pq - p
    if(q == 1):
        fd = fd_intq1(pq,a)
    elif(q == 2):
        fd = fd_intq2(pq,a)
    elif(q == 3):
        fd = fd_intq3(pq,a)
    elif(q == 4):
        fd = fd_intq4(pq,a)
    else:
        print("the resonance must be 4th or lower order")
        return
    return fd



def fd_ext(pq,p):
    a = np.power((float(p)/float(pq)),(2./3))
    q = pq - p
    if(q == 1):
        fd = fd_extq1(-p,a)
    elif(q == 2):
        fd = fd_extq2(-p,a)
    elif(q == 3):
        fd = fd_extq3(-p,a)
    elif(q == 4):
        fd = fd_extq4(-p,a)
    else:
        print("the resonance must be 4th or lower order")
        return
    return fd



##############################################################################
# Disturbing Function coefficients

#internal 1st order resonance
def fd_intq1(pq,a):
    j = float(pq)
    t1 = -2.*j*b(j,0.5,a)
    t2 = -a*Db(j,0.5,a)
    val = 0.5*(t1+t2)
    return val

#external 1st order resonance
def fd_extq1(pq,a):
    j = float(pq)
    t1 = (-1. + 2.*j)*b(j-1.,0.5,a)
    t2 = -a*Db(j-1.,0.5,a)
    val = 0.5*(t1+t2)
    return val

#internal 2nd order resonance
def fd_intq2(pq,a):
    j = float(pq)
    t1 = (-5.*j + 4.*j*j)*b(j,0.5,a)
    t2 = (-2.*a + 4.*j*a)*Db(j,0.5,a)
    t3 = (a*a)*Dnb(2,j,0.5,a)    
    val = (1./8.)*(t1+t2+t3)
    return val

#external 2nd order resonance
def fd_extq2(pq,a):
    j = float(pq)
    t1 = (2. - 7.*j + 4.*j*j)*b(j-2.,0.5,a)
    t2 = (-2.*a + 4.*j*a)*Db(j-2.,0.5,a)
    t3 = (a*a)*Dnb(2,j-2.,0.5,a)    
    val = (1./8.)*(t1+t2+t3)
    return val

#internal 3rd order resonance
def fd_intq3(pq,a):
    j = float(pq)
    t1 = (-26.*j + 30.*j*j - 8.*j*j*j)*b(j,0.5,a)
    t2 = (-9.*a + 27.*j*a - 12.*j*j*a)*Db(j,0.5,a)
    t3 = (6.*a*a - 6.*a*a*j)*Dnb(2,j,0.5,a)
    t4 = -a*a*a*Dnb(3,j,0.5,a)
    val = (1./48.)*(t1+t2+t3+t4)
    return val

#external 3rd order resonance
def fd_extq3(pq,a):
    j = float(pq)
    t1 = (-6. + 29.*j - 30.*j*j + 8.*j*j*j)*b(j-3.,0.5,a)
    t2 = (6.*a - 21.*j*a + 12.*j*j*a)*Db(j-3.,0.5,a)
    t3 = (-3.*a*a + 6.*a*a*j)*Dnb(2,j-3.,0.5,a)
    t4 = a*a*a*Dnb(3,j-3.,0.5,a)
    val = (1./48.)*(t1+t2+t3+t4)
    return val

#internal 4th order resonance
def fd_intq4(pq,a):
    j = float(pq)
    t1 = (-206.*j + 283.*j*j - 120.*j*j*j + 16.*j*j*j*j)*b(j,0.5,a)
    t2 = (-64.*a +236.*j*a - 168.*j*j*a + 32.*j*j*j*a)*Db(j,0.5,a)
    t3 = (48.*a*a - 78.*a*a*j + 24.*j*j*a*a)*Dnb(2,j,0.5,a)
    t4 = (-12.*a*a*a +8*a*a*a*j)*Dnb(3,j,0.5,a)
    t5 = (a*a*a*a)*Dnb(4,j,0.5,a)
    val = (1./384.)*(t1+t2+t3+t4+t5)
    return val

#external 4th order resonance
def fd_extq4(pq,a):
    j = float(pq)
    t1 = (24. - 146.*j + 211.*j*j - 104.*j*j*j + 16.*j*j*j*j)*b(j-4.,0.5,a)
    t2 = (-24.*a +116.*j*a - 120.*j*j*a + 32.*j*j*j*a)*Db(j-4.,0.5,a)
    t3 = (12.*a*a - 42.*a*a*j + 24.*j*j*a*a)*Dnb(2,j-4.,0.5,a)
    t4 = (-4.*a*a*a +8*a*a*a*j)*Dnb(3,j-4.,0.5,a)
    t5 = (a*a*a*a)*Dnb(4,j-4.,0.5,a)
    val = (1./384.)*(t1+t2+t3+t4+t5)
    return val
